### A2: Optimization with Genetic Algorithms

#### Name: Maria Alejandra Mamani Cordero

### Imports and Setup

In [ ]:
# Basic libraries
import random
import numpy as np

# Libraries for graph and visualization
import networkx as nx
import matplotlib.pyplot as plt
